In [ ]:
import pickle
import random
import time
from pathlib import Path

import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix

import pyranges as pr
from pyranges.pyranges_main import PyRanges

from utils import *
from peak_parser import *


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  #ban GPU

import tensorflow as tf
import data_io

In [ ]:
#Custom TF peak length and epigenomic region
#Extract DNA sequence at TF bingding sites (200bp)
#
cell='GM12878'
tranf='CTCF'

chroms=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrX','chrY',]
genome_fa_file='data/genome/GRCh38.primary_assembly.genome.fa' #downloaded from https://www.gencodegenes.org/

path_peak='data/training'
path_cell=path_peak+'/'+cell
path_tf=path_cell+'/'+tranf
tf_file=path_cell+'/'+tranf+'.bed'

reg_file=path_tf+'/'+tranf+'_40k.bed'
bed_file=path_tf+'/'+tranf+'_200.bed'
reg_file_null=path_tf+'/'+tranf+'_null_40k.bed'
bed_file_null=path_tf+'/'+tranf+'_null_200.bed'
seq_file=path_tf+'/'+tranf+'_200.fa'
seq_file_null=path_tf+'/'+tranf+'_null_200.fa'

custom_bed(peak_file=tf_file,reg_file=reg_file,bed_file=bed_file,reg_file_null=reg_file_null,bed_file_null=bed_file_null,chroms=chroms)
extract_fasta(bed_file=bed_file,fa_file=genome_fa_file,seq_file=seq_file)
extract_fasta(bed_file=bed_file_null,fa_file=genome_fa_file,seq_file=seq_file_null)


In [ ]:
#Negative FIMO sites，generated by run_fimo.sh
#
fimo_neg_file=path_tf+'/'+'fimo_neg.bed'

bedfile_fimo=path_tf+'/'+'fimo_neg_200.bed'
regfile_fimo=path_tf+'/'+'fimo_neg_40k.bed'
seqfile_fimo=path_tf+'/'+'fimo_neg_200.fa'

fimo_neg=pr.read_bed(fimo_neg_file)
fimo_neg = fimo_neg[fimo_neg.Chromosome.isin(chroms)]

bed_region_fimo=custom_peak(fimo_neg,length=40000,write_file=regfile_fimo)
custom_peak(bed_region_fimo,length=200,write_file=bedfile_fimo)
extract_fasta(bed_file=bedfile_fimo,fa_file=genome_fa_file,seq_file=seqfile_fimo)
            

In [ ]:
# Convert epigenomic information (EI) to state values (0,1) 
#
epi_targets=['ATAC', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9me3']  #

peak_files={x:path_cell+'/'+x+'.bed' for x in epi_targets}
ohpeak_files={x:path_cell+'/'+x+'.bed.pickle' for x in epi_targets}

for k,v in peak_files.items():
    if Path(v).exists():
        print(v)
        peak_2onehot_chrom_whole(v,ohpeak_files[k])
    else:
        print(v,'no such file.')

In [ ]:
#Integrate DNA info. and EI info.
#
import shutil

tmpdir=path_peak+'/'+'tmp'
Path(tmpdir).mkdir(exist_ok=True)

seq_file_pos=path_tf+'/'+tranf+'_200.fa'
reg_file_pos=path_tf+'/'+tranf+'_40k.bed'
sent_file_pos=path_tf+'/'+tranf+'_200.pickle'
            
seq_file_null=path_tf+'/'+tranf+'_null_200.fa'
reg_file_null=path_tf+'/'+tranf+'_null_40k.bed'
sent_file_null=path_tf+'/'+tranf+'_null_200.pickle'

seq_file_neg=path_tf+'/'+'fimo_neg_200.fa'
reg_file_neg=path_tf+'/'+'fimo_neg_40k.bed'
sent_file_neg=path_tf+'/'+'fimo_neg_200.pickle'

#Peak sites and null (random) sites
generate_peak_context(seq_file=seq_file_pos,reg_file=reg_file_pos,label=1,targets=epi_targets,targets_files=ohpeak_files,out_file=sent_file_pos,tmpdir=tmpdir)
generate_peak_context(seq_file=seq_file_null,reg_file=reg_file_null,label=0,targets=epi_targets,targets_files=ohpeak_files,out_file=sent_file_null,tmpdir=tmpdir)
#Negative FIMO sites
generate_peak_context(seq_file=seq_file_neg,reg_file=reg_file_neg,label=0,targets=epi_targets,targets_files=ohpeak_files,out_file=sent_file_neg,tmpdir=tmpdir)

shutil.rmtree(tmpdir) 


In [ ]:
#Build tfrecord dataset, use all EI info.
#
sent_file_pos=path_tf+'/'+tranf+'_200.pickle'
sent_file_null=path_tf+'/'+tranf+'_null_200.pickle'
sent_file_neg=path_tf+'/'+'fimo_neg_200.pickle'

file_peak_set=path_tf+'/'+'peak_set.tfrecord'
file_train_set=path_tf+'/'+'train_set.tfrecord'
file_test_set=path_tf+'/'+'test_set.tfrecord'
    
peak_set=data_io.select_sample(sample_file=sent_file_pos)  #no shuffling
data_io.write_tfrecord(peak_set,file_peak_set) 

dataset=[]
dataset+=data_io.select_sample(sample_file=sent_file_pos)
dataset+=data_io.select_sample(sample_file=sent_file_null)
dataset+=data_io.select_sample(sample_file=sent_file_neg)
    
random.shuffle(dataset)  #shuffle 
train_set,test_set=data_io.separate_dataset(dataset)  #
    
data_io.write_tfrecord(train_set,file_train_set)
data_io.write_tfrecord(test_set,file_test_set)


In [ ]:
#Build tfrecord dataset, use select EI info.
epi_targets_comb={
    'epi-only':'ATAC_H3K27ac_H3K27me3_H3K36me3_H3K4me1_H3K4me3_H3K9me3',
    'dna-only':'',
    'atac':'ATAC',
    'h3k27ac':'H3K27ac',
    'h3k27me3':'H3K27me3',
    'h3k36me3':'H3K36me3',
    'h3k4me1':'H3K4me1',
    'h3k4me3':'H3K4me3',
    'h3k9me3':'H3K9me3',
    'bivalent':'H3K27me3_H3K4me3',
    'epi-core':'H3K27me3_H3K36me3_H3K4me1_H3K4me3_H3K9me3'
}

In [ ]:
# Taking ATAC as an example
#
target='atac'

sent_file_pos=path_tf+'/'+tranf+'_200.pickle'
sent_file_null=path_tf+'/'+tranf+'_null_200.pickle'
sent_file_neg=path_tf+'/'+'fimo_neg_200.pickle'

target_epis=epi_targets_comb[target].split('_')
keep_dna=False if target=='epi-only' else True
        
file_peak_set=path_tf+'/'+'peak_set_'+target+'.tfrecord'
file_train_set=path_tf+'/'+'train_set_'+target+'.tfrecord'
file_test_set=path_tf+'/'+'test_set_'+target+'.tfrecord'

peak_set=data_io.select_sample(sample_file=sent_file_pos)  #No shuffling
peak_set=[mask_peak_context(x,target_epis,keep_dna=keep_dna) for x in peak_set]
data_io.write_tfrecord(peak_set,file_peak_set)
        
dataset=[]
dataset+=data_io.select_sample(sample_file=sent_file_pos)
dataset+=data_io.select_sample(sample_file=sent_file_null)
dataset+=data_io.select_sample(sample_file=sent_file_neg)
random.shuffle(dataset)

dataset=[mask_peak_context(x,target_epis,keep_dna=keep_dna) for x in dataset]
train_set,test_set=data_io.separate_dataset(dataset)
data_io.write_tfrecord(train_set,file_train_set)
data_io.write_tfrecord(test_set,file_test_set)
